This version specifies a random portion of missingness in the features. It is identical with missing_cleanml-2-percentage. These two notebooks are used to run datasets in parallel. Airbnb in particular takes a long time to run.

In [1]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import utils
import pickle
from collections import defaultdict
import numpy as np
from collections import OrderedDict
from sklearn.metrics import f1_score 
import pandas as pd
from sklearn.metrics import f1_score 

/mnt/ds3lab-scratch/raox/gatech_data_cleaning/ICDE2021-revision/nacl36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/mnt/ds3lab-scratch/raox/gatech_data_cleaning/ICDE2021-revision/nacl36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import sys
sys.setrecursionlimit(10**6)

In [3]:
import math
import matplotlib.pyplot as plt 
from sklearn.datasets import load_digits
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pylab
import sys
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
import keras
import time
import pandas as pd
import json
import operator

Using TensorFlow backend.


In [4]:
sys.path.append("../NaCL/code/")
import utils
import LR2NB_GP
sys.path.append("../NaCL/notebooks/")
from dataload import *

In [5]:
from preprocess_mv import preprocess_data

/mnt/ds3lab-scratch/raox/gatech_data_cleaning/ICDE2021-revision/nacl36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [6]:
%load_ext autoreload
%autoreload 2

# Loading Data

In [7]:
seed = 3659

In [8]:
data_dir = "data-robustml-mv"

In [9]:
save_dir = "result-robustml"
model_name ='nacl'

In [10]:
accuracy = lambda x,y: np.mean(x == y)
f1 = lambda x,y: f1_score(x,y.flatten(), average = "weighted")



In [11]:
dataset = 'USCensus'

In [12]:
jobs = get_jobs_mv(data_dir, dataset = [dataset])
print("Number of jobs:", len(jobs))

Number of jobs: 20


In [13]:
jobs

[['data-robustml-mv', 'USCensus', 'missing_values', 'split_9394'],
 ['data-robustml-mv', 'USCensus', 'missing_values', 'split_8444'],
 ['data-robustml-mv', 'USCensus', 'missing_values', 'split_6542'],
 ['data-robustml-mv', 'USCensus', 'missing_values', 'split_235'],
 ['data-robustml-mv', 'USCensus', 'missing_values', 'split_7751'],
 ['data-robustml-mv', 'USCensus', 'missing_values', 'split_4764'],
 ['data-robustml-mv', 'USCensus', 'missing_values', 'split_2895'],
 ['data-robustml-mv', 'USCensus', 'missing_values', 'split_5396'],
 ['data-robustml-mv', 'USCensus', 'missing_values', 'split_5192'],
 ['data-robustml-mv', 'USCensus', 'missing_values', 'split_8093'],
 ['data-robustml-mv', 'USCensus', 'missing_values', 'split_2962'],
 ['data-robustml-mv', 'USCensus', 'missing_values', 'split_144'],
 ['data-robustml-mv', 'USCensus', 'missing_values', 'split_905'],
 ['data-robustml-mv', 'USCensus', 'missing_values', 'split_2516'],
 ['data-robustml-mv', 'USCensus', 'missing_values', 'split_7813']

In [14]:
accuracy = lambda x,y: np.mean(x == y)
f1 = lambda x,y: f1_score(x,y.flatten(), average = "weighted")



In [15]:
with open('/mnt/ds3lab-scratch/raox/gatech_data_cleaning/ICDE2021-revision/CleanML/result/{}_result.json'.format(dataset)) as json_file:
    results = json.load(json_file)

In [16]:
%%time 

for job in jobs:
    
    print("Processing", job)
    print('--------------')
    dataset = job[1]

    dirty_X_train = pd.read_csv(os.path.join(*job, "dirty_X_train.csv"))
    dirty_y_train = pd.read_csv(os.path.join(*job, "dirty_y_train.csv"))
    dirty_X_test = pd.read_csv(os.path.join(*job, "dirty_X_test.csv"))
    dirty_y_test = pd.read_csv(os.path.join(*job, "dirty_y_test.csv"))

    dirty_X_train = dirty_X_train.fillna(0)
    dirty_X_test = dirty_X_test.fillna(0)
    X_train, y_train, X_test, y_test, mappings, mappings_dict = preprocess_data(dataset, 
                                                             dirty_X_train, dirty_y_train, 
                                                             [dirty_X_test], [dirty_y_test])
    X_test = X_test[0]
    y_test = y_test[0]
    dirty_X_test = pd.read_csv(os.path.join(*job, "dirty_X_test.csv"))
    null_data = dirty_X_test[dirty_X_test.isnull().any(axis=1)]

    missing = [[]] * dirty_X_test.shape[0]
    for i, row in null_data.iterrows():
        # append the original index of the missing feature
        cols = null_data.columns.get_values()[np.where(pd.isnull(row))]

        idx = [mappings_dict[col][0] for col in cols]
        missing[i] = idx
    missing = np.array(missing)

    output = defaultdict(lambda: defaultdict(list))

    output['train']['nacl'] = (X_train, y_train)
    output['test']['nacl'] = (X_test, y_test)
    output['missing_test'] = missing
    output['mappings1hot_dict'] = mappings_dict
    output['mappings1hot_list'] = mappings
    pickle.dump(dict(output), open(os.path.join(*job,"data.p"), 'wb'))


    max_acc = 0

    for method in ['delete', 'impute_holoclean', 'impute_mean_mode', 'impute_mean_dummy', 
                   'impute_median_mode', 'impute_median_dummy', 'impute_mode_mode', 'impute_mode_dummy']:
        #print('Titanic/v{}/missing_values/{}/logistic_regression/3659'.\
        #           format(split_seed, method))
        split_seed = job[-1].split("_")[1]
        a =  results['{}/v{}/missing_values/{}/logistic_regression/3659'.\
                   format(dataset, split_seed, method)].copy()
        #print(a)
        del a['best_params']
        del a['train_acc']
        del a['val_acc']
        max_key = max(a.items(), key=operator.itemgetter(1))[0]

        if a[max_key] > max_acc:
            best_params_lr = results['{}/v{}/missing_values/{}/logistic_regression/3659'.\
                   format(dataset, split_seed, method)]['best_params']
            max_acc = a[max_key]

    clf = LogisticRegression(solver='lbfgs', 
    verbose=True, 
    C = best_params_lr['C'],
    max_iter=2000, n_jobs=-1).fit(X_train, y_train)
    lr_pred = clf.predict(X_test)
    lr_train_acc = accuracy(clf.predict(X_train), y_train)
    lr_test_acc = accuracy(lr_pred, y_test)
    lr_test_f1 = f1(lr_pred, y_test)

    print("LR Train Accuracy: {} \nLR Test  Accuracy: {}".format(lr_train_acc, lr_test_acc))

    W = np.hstack((clf.intercept_[:,None], clf.coef_))[0]
    nacl = LR2NB_GP.LR2NB(W)
    nacl.setObj(X_train, y_train)

    # Using Mosek Solver
    nacl.solve(verbose=1)

    # Sanity Check to compare NaCL results with Logistic Regression
    assert(1.0 == np.average( nacl.classify(X_test)  == clf.predict(X_test )))
    assert(1.0 == np.average( nacl.classify(X_train) == clf.predict(X_train)))
    nacl_pred = nacl.classify(X_test)
    nacl_train_acc = accuracy(nacl.classify(X_train), y_train)
    nacl_test_acc = accuracy(nacl_pred, y_test)
    #print(nacl_pred)
    #print(y_test)
    nacl_test_f1 = f1(nacl_pred[0], y_test)

    print("NaCL Train Accuracy: {} \nNaCL Test  Accuracy: {}".format(nacl_train_acc, nacl_test_acc))

    NB = BernoulliNB().fit(X_train, y_train)
    NB_pred = NB.predict(X_test)
    nb_train_acc = accuracy(NB.predict(X_train), y_train)
    nb_test_acc = accuracy(NB_pred, y_test)
    nb_test_f1 = f1(NB_pred, y_test)

    print("NB Train Accuracy: {} \nNB Test  Accuracy: {}".format(nb_train_acc, nb_test_acc))

    # Make an example observations with some missing features

    # setting the dimensions to be zero
    missing_mask = np.zeros(X_test.shape) 


    miss = missing
    for i in range(len(missing_mask)):
        for j in range(len(miss[i])):
            missing_mask[i][mappings[miss[i][j]]] = 1

    a = nacl.predict_proba(X_test, missing = missing_mask)[:,1]
    a = np.where(a > 0.5, 1, 0)

    test_acc = accuracy(y_test, a)
    test_f1 = f1(y_test, a)
    print('nacl test acc with missing:', test_acc)
    print('nacl test f1 with missing:', test_f1) # true, pred


    result_dict = {}

    result_dict["best_params"] = best_params_lr


    result_dict["{}_test_f1".format('nacl_missing')] = test_f1
    result_dict["{}_test_acc".format('nacl_missing')] = test_acc

    result_dict["{}_test_f1".format('nacl')] = nacl_test_f1
    result_dict["{}_test_acc".format('nacl')] = nacl_test_acc
    result_dict["{}_train_acc".format('nacl')] = nacl_train_acc

    result_dict["{}_test_f1".format('nb')] = nb_test_f1
    result_dict["{}_test_acc".format('nb')] = nb_test_acc
    result_dict["{}_train_acc".format('nb')] = nb_train_acc

    result_dict["{}_test_f1".format('lr')] = lr_test_f1
    result_dict["{}_test_acc".format('lr')] = lr_test_acc
    result_dict["{}_train_acc".format('lr')] = lr_train_acc

    print(result_dict)
    save_result_mv(job, seed, result_dict, model_name, save_dir)





Processing ['data-robustml-mv', 'USCensus', 'missing_values', 'split_9394']
--------------
dataset: {'data_dir': 'USCensus', 'error_types': ['missing_values'], 'label': 'Income', 'ml_task': 'classification'}
columns to encode: ['Workclass', 'Education', 'Marital-status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Native-country']


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    5.3s finished


LR Train Accuracy: 0.8550476190476191 
LR Test  Accuracy: 0.8477777777777777
Using solver 'cvxopt'
 for 434 free variables
  in 436 posynomial inequalities.
Solving took 4.07 seconds.
NaCL Train Accuracy: 0.8550476190476191 
NaCL Test  Accuracy: 0.8477777777777777
NB Train Accuracy: 0.7874285714285715 
NB Test  Accuracy: 0.7886666666666666
nacl test acc with missing: 0.8477777777777777
nacl test f1 with missing: 0.8430716362799788
{'best_params': {'C': 1.7971880102761828}, 'nacl_missing_test_f1': 0.8430716362799788, 'nacl_missing_test_acc': 0.8477777777777777, 'nacl_test_f1': 0.8524839192755768, 'nacl_test_acc': 0.8477777777777777, 'nacl_train_acc': 0.8550476190476191, 'nb_test_f1': 0.7782574790265708, 'nb_test_acc': 0.7886666666666666, 'nb_train_acc': 0.7874285714285715, 'lr_test_f1': 0.8524839192755768, 'lr_test_acc': 0.8477777777777777, 'lr_train_acc': 0.8550476190476191}
Processing ['data-robustml-mv', 'USCensus', 'missing_values', 'split_8444']
--------------
dataset: {'data_dir':

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    2.4s finished


LR Train Accuracy: 0.8467619047619047 
LR Test  Accuracy: 0.8473333333333334
Using solver 'cvxopt'
 for 434 free variables
  in 436 posynomial inequalities.
Solving took 6.41 seconds.
NaCL Train Accuracy: 0.8467619047619047 
NaCL Test  Accuracy: 0.8473333333333334
NB Train Accuracy: 0.7847619047619048 
NB Test  Accuracy: 0.778
nacl test acc with missing: 0.8471111111111111
nacl test f1 with missing: 0.8411307058358066
{'best_params': {'C': 1286.4014641512479}, 'nacl_missing_test_f1': 0.8411307058358066, 'nacl_missing_test_acc': 0.8471111111111111, 'nacl_test_f1': 0.8532717886370916, 'nacl_test_acc': 0.8473333333333334, 'nacl_train_acc': 0.8467619047619047, 'nb_test_f1': 0.7660630573748298, 'nb_test_acc': 0.778, 'nb_train_acc': 0.7847619047619048, 'lr_test_f1': 0.8532717886370916, 'lr_test_acc': 0.8473333333333334, 'lr_train_acc': 0.8467619047619047}
Processing ['data-robustml-mv', 'USCensus', 'missing_values', 'split_6542']
--------------
dataset: {'data_dir': 'USCensus', 'error_types'

...

LR Train Accuracy: 0.8501904761904762 
LR Test  Accuracy: 0.8553333333333333
Using solver 'cvxopt'
 for 430 free variables
  in 432 posynomial inequalities.
Solving took 5.66 seconds.
NaCL Train Accuracy: 0.8501904761904762 
NaCL Test  Accuracy: 0.8553333333333333
NB Train Accuracy: 0.7750476190476191 
NB Test  Accuracy: 0.7917777777777778
nacl test acc with missing: 0.8564444444444445
nacl test f1 with missing: 0.8493607491481172
{'best_params': {'C': 1286.4014641512479}, 'nacl_missing_test_f1': 0.8493607491481172, 'nacl_missing_test_acc': 0.8564444444444445, 'nacl_test_f1': 0.8622409343980176, 'nacl_test_acc': 0.8553333333333333, 'nacl_train_acc': 0.8501904761904762, 'nb_test_f1': 0.7809948017680072, 'nb_test_acc': 0.7917777777777778, 'nb_train_acc': 0.7750476190476191, 'lr_test_f1': 0.8622409343980176, 'lr_test_acc': 0.8553333333333333, 'lr_train_acc': 0.8501904761904762}
Processing ['data-robustml-mv', 'USCensus', 'missing_values', 'split_5056']
--------------
dataset: {'data_dir':

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.5s finished


LR Train Accuracy: 0.85 
LR Test  Accuracy: 0.8446666666666667
Using solver 'cvxopt'
 for 430 free variables
  in 432 posynomial inequalities.
Solving took 4.71 seconds.
NaCL Train Accuracy: 0.85 
NaCL Test  Accuracy: 0.8446666666666667
NB Train Accuracy: 0.7805714285714286 
NB Test  Accuracy: 0.7857777777777778
nacl test acc with missing: 0.844
nacl test f1 with missing: 0.8377440024745696
{'best_params': {'C': 1286.4014641512479}, 'nacl_missing_test_f1': 0.8377440024745696, 'nacl_missing_test_acc': 0.844, 'nacl_test_f1': 0.8508615049562119, 'nacl_test_acc': 0.8446666666666667, 'nacl_train_acc': 0.85, 'nb_test_f1': 0.7741800607291753, 'nb_test_acc': 0.7857777777777778, 'nb_train_acc': 0.7805714285714286, 'lr_test_f1': 0.8508615049562119, 'lr_test_acc': 0.8446666666666667, 'lr_train_acc': 0.85}
Processing ['data-robustml-mv', 'USCensus', 'missing_values', 'split_3562']
--------------
dataset: {'data_dir': 'USCensus', 'error_types': ['missing_values'], 'label': 'Income', 'ml_task': 'cla

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.4s finished


LR Train Accuracy: 0.8508571428571429 
LR Test  Accuracy: 0.8493333333333334
Using solver 'cvxopt'
 for 430 free variables
  in 432 posynomial inequalities.
Solving took 2.51 seconds.
NaCL Train Accuracy: 0.8508571428571429 
NaCL Test  Accuracy: 0.8493333333333334
NB Train Accuracy: 0.7841904761904762 
NB Test  Accuracy: 0.7786666666666666
nacl test acc with missing: 0.8482222222222222
nacl test f1 with missing: 0.8430178904608019
{'best_params': {'C': 0.7185624359789785}, 'nacl_missing_test_f1': 0.8430178904608019, 'nacl_missing_test_acc': 0.8482222222222222, 'nacl_test_f1': 0.8547882556546577, 'nacl_test_acc': 0.8493333333333334, 'nacl_train_acc': 0.8508571428571429, 'nb_test_f1': 0.7665868183895871, 'nb_test_acc': 0.7786666666666666, 'nb_train_acc': 0.7841904761904762, 'lr_test_f1': 0.8547882556546577, 'lr_test_acc': 0.8493333333333334, 'lr_train_acc': 0.8508571428571429}
CPU times: user 3min 29s, sys: 9min 5s, total: 12min 34s
Wall time: 2min 59s
